In [34]:
import os
import dotenv
import time
import duckdb

from duckdb import DuckDBPyConnection
import polars as pl
from deltalake import DeltaTable, write_deltalake
from polars import DataFrame, LazyFrame
from pyarrow.dataset import dataset

from s3fs import S3FileSystem
from script.analysis import tweak_result

In [35]:
def duckdb_connection() -> DuckDBPyConnection:
    """Connect to DuckDb and set up some additional extensions

    Returns:
        DuckDBPyConnection: A DuckDB python connection
    """    
    
    # loat .env file
    dotenv.load_dotenv()

    # connect to duckdb and setup extensions
    con = duckdb.connect()
    con.sql(
        f"""
        INSTALL httpfs;
        LOAD httpfs;
        PRAGMA enable_optimizer;
        SET s3_region='{os.getenv('AWS_DEFAULT_REGION')}';
        SET s3_access_key_id={os.getenv('AWS_ACCESS_KEY_ID')};
        SET s3_secret_access_key='{os.getenv('AWS_SECRET_ACCESS_KEY')}';
        """
    )
    return con


def load_data_to_deltatable(con: DuckDBPyConnection):
    """Load data and transform to deltatable.
    This function also create a table inside database just in case for other purposes

    Args:
        con (DuckDBPyConnection): a DuckDB Python connection 
    """    
    
    con.sql(
        """
        CREATE OR REPLACE TABLE march_delivery AS 
        SELECT 
            *
        FROM read_parquet('s3://sonlebucket/data/delivery_orders_march.parquet');
        """
    )

    arrow_table = con.sql(
        """
            SELECT * FROM march_delivery;
            """
    ).arrow()

    write_deltalake(
        data=arrow_table,
        table_or_uri="march_order",
        mode="overwrite",
        overwrite_schema=True,
    )


In [36]:
con = duckdb_connection()
load_data_to_deltatable(con)

con.sql("select count(orderid) from march_delivery").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────┐
│ count(orderid) │
│     int64      │
├────────────────┤
│        3176313 │
└────────────────┘



In [37]:
con.sql("SUMMARIZE march_delivery")

┌─────────────────────┬─────────────┬──────────────────────┬───┬────────────────────┬─────────┬─────────────────┐
│     column_name     │ column_type │         min          │ … │        q75         │  count  │ null_percentage │
│       varchar       │   varchar   │       varchar        │   │      varchar       │  int64  │     varchar     │
├─────────────────────┼─────────────┼──────────────────────┼───┼────────────────────┼─────────┼─────────────────┤
│ orderid             │ BIGINT      │ 2195009611           │ … │ 2337318984         │ 3176313 │ 0.0%            │
│ pick                │ BIGINT      │ 1583079143           │ … │ 1584502740         │ 3176313 │ 0.0%            │
│ 1st_deliver_attempt │ DOUBLE      │ 1583131545.0         │ … │ 1584938645.7577686 │ 3176313 │ 0.0%            │
│ 2nd_deliver_attempt │ DOUBLE      │ 1583251305.0         │ … │ 1584992711.4294174 │ 3176313 │ 57.28%          │
│ buyeraddress        │ VARCHAR     │ 									 #227C TNHS condom…  │ … │ NULL          

In [38]:
dt = DeltaTable("march_order")
print(dt.files())
dt.schema().to_pyarrow()
# dt.optimize.z_order(['orderid', 'pick'])

['0-5d681f95-bdf3-4aac-bc69-44029205e379-0.parquet']


orderid: int64
pick: int64
1st_deliver_attempt: double
2nd_deliver_attempt: double
buyeraddress: string
selleraddress: string

In [39]:
dt.optimize.compact()
dt.optimize.z_order(['orderid', 'pick'])

{'numFilesAdded': 4,
 'numFilesRemoved': 1,
 'filesAdded': {'min': 3423418,
  'max': 111864641,
  'avg': 84411001.25,
  'totalFiles': 4,
  'totalSize': 337644005},
 'filesRemoved': {'min': 586475920,
  'max': 586475920,
  'avg': 586475920.0,
  'totalFiles': 1,
  'totalSize': 586475920},
 'partitionsOptimized': 0,
 'numBatches': 1,
 'totalConsideredFiles': 1,
 'totalFilesSkipped': 0,
 'preserveInsertionOrder': True}

In [40]:
df = pl.scan_delta("march_order")
result = tweak_result(df).collect()
result

is_late,count_order,percent_slate
i32,u32,f64
0,2413891,0.759966
1,762422,0.240034


In [ ]:
con.sql("select * from result;").to_table("result")

con.sql("copy result to 's3://sonlebucket/data/result.parquet' (format 'parquet')")

In [ ]:
dt.vacuum(retention_hours=0, enforce_retention_duration=False, dry_run=False)

In [ ]:
def read_s3_parquet(s3_loc) -> LazyFrame | None:
    # check .env files

    # setup cloud filesystem access
    try:
        cloudfs = S3FileSystem(
            key=os.getenv("AWS_ACCESS_KEY_ID"),
            secret=os.getenv("AWS_SECRET_ACCESS_KEY"),
        )

        # reference multiple parquet files
        pyarrow_dataset = dataset(
            source=s3_loc,
            filesystem=cloudfs,
            format="parquet",
        )

        # load efficiently into polars
        df = pl.scan_pyarrow_dataset(pyarrow_dataset)
        return df
    except Exception as e:
        print(f"Error reading parquet file from AMZ S3: {e}")
        return None

In [ ]:
s3_loc = "s3://sonlebucket/data/delivery_orders_march.parquet"
cloudfs = S3FileSystem(
    profile="sonlebucket",
)

# reference multiple parquet files
pyarrow_dataset = dataset(
    source=s3_loc,
    filesystem=cloudfs,
    format="parquet",
)

# load efficiently into polars
df = pl.scan_pyarrow_dataset(pyarrow_dataset)

In [ ]:
con.write